### Imports and Downloads

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
# Step 4: Download dataset
!kaggle datasets download -d franciscoescobar/satellite-images-of-water-bodies

In [ ]:
# Step 5: Unzip the dataset
!unzip satellite-images-of-water-bodies.zip

### Data Processing

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
import gc
from tqdm import tqdm

# Define paths to your dataset
IMAGE_PATH = '/content/Water Bodies Dataset/Images'
MASK_PATH = '/content/Water Bodies Dataset/Masks'
IMAGE_SIZE = (256, 256)  # Resize all images to 256x256

# Helper function to load image and mask names from the text files
def load_file_list(file_path):
    with open(file_path, 'r') as f:
        file_list = f.read().splitlines()
    return file_list

# Load image names from the train.txt, val.txt, and test.txt files
train_image_list = load_file_list('train.txt')
val_image_list = load_file_list('val.txt')
test_image_list = load_file_list('test.txt')

# Append the full path to the image and mask names
def append_paths(image_list, image_path, mask_path):
    images = [os.path.join(image_path, img) for img in image_list]
    masks = [os.path.join(mask_path, img) for img in image_list]
    return images, masks

# Create the full paths for train, val, and test images and masks
image_train, mask_train = append_paths(train_image_list, IMAGE_PATH, MASK_PATH)
image_val, mask_val = append_paths(val_image_list, IMAGE_PATH, MASK_PATH)
image_test, mask_test = append_paths(test_image_list, IMAGE_PATH, MASK_PATH)

# Print the number of train, validation, and test images and masks
num_train = len(image_train)
num_val = len(image_val)
num_test = len(image_test)

print(f"Number of training images: {num_train}")
print(f"Number of validation images: {num_val}")
print(f"Number of test images: {num_test}")

# Calculate and print the percentage splits
total_images = num_train + num_val + num_test
train_split = (num_train / total_images) * 100
val_split = (num_val / total_images) * 100
test_split = (num_test / total_images) * 100

print(f"Train split: {train_split:.2f}%")
print(f"Validation split: {val_split:.2f}%")
print(f"Test split: {test_split:.2f}%")

# Function to resize and load images in batches
def data_generator(image_files, mask_files, batch_size, image_size):
    while True:
        for i in range(0, len(image_files), batch_size):
            batch_image_files = image_files[i:i + batch_size]
            batch_mask_files = mask_files[i:i + batch_size]

            images = []
            masks = []

            for img_file, mask_file in zip(batch_image_files, batch_mask_files):
                # Load the image and mask
                img = cv2.imread(img_file)
                mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)

                # Resize both image and mask
                img = cv2.resize(img, image_size)
                mask = cv2.resize(mask, image_size)

                # Normalize image and binary mask
                img = img / 255.0
                mask = mask / 255.0

                images.append(img)
                masks.append(np.expand_dims(mask, axis=-1))  # Add an extra dimension for the mask

            yield np.array(images), np.array(masks)

# Parameters for data generator
BATCH_SIZE = 8

# Create data generators
train_gen = data_generator(image_train, mask_train, BATCH_SIZE, IMAGE_SIZE)
val_gen = data_generator(image_val, mask_val, BATCH_SIZE, IMAGE_SIZE)
test_gen = data_generator(image_test, mask_test, BATCH_SIZE, IMAGE_SIZE)

### Model Definition and Training

In [ ]:
# Define the U-Net architecture
def unet_model(input_size):
    inputs = Input(input_size)

    # Encoding path (Contracting)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoding path (Expansive)
    up6 = UpSampling2D(size=(2, 2))(drop5)
    up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs, outputs)

    # Compile the U-Net model with additional metrics
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', Precision(), Recall(), MeanIoU(num_classes=2)])

    return model

# Create the U-Net model
input_size = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
model = unet_model(input_size)

# Display the model summary
model.summary()

# Define callbacks
#earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint('unet_water_segmentation.keras', verbose=1, save_best_only=True)

# Train the model using data generators
steps_per_epoch = len(image_train) // BATCH_SIZE
validation_steps = len(image_val) // BATCH_SIZE

# Train the model using data generators
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=50,
    #callbacks=[earlystopper, checkpointer]
    callbacks=[checkpointer]
)

#To download the model, but checkpoint should be saved eitherways
#from google.colab import files
#files.download('unet_water_segmentation.keras')

### Results EDA

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Get the number of epochs
    epochs = range(1, len(history.history['accuracy']) + 1)

    # Specify tick values at intervals of 5, starting from 1
    tick_values = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45,50]

    plt.figure(figsize=(12, 8))

    # Plot Accuracy
    plt.subplot(2, 2, 1)
    plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(tick_values)  # Set x-ticks to the desired specific values
    plt.legend()

    # Plot Precision
    plt.subplot(2, 2, 2)
    plt.plot(epochs, history.history['precision'], label='Training Precision')
    plt.plot(epochs, history.history['val_precision'], label='Validation Precision')
    plt.title('Model Precision')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.xticks(tick_values)  # Set x-ticks to the desired specific values
    plt.legend()

    # Plot Recall
    plt.subplot(2, 2, 3)
    plt.plot(epochs, history.history['recall'], label='Training Recall')
    plt.plot(epochs, history.history['val_recall'], label='Validation Recall')
    plt.title('Model Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.xticks(tick_values)  # Set x-ticks to the desired specific values
    plt.legend()

    # Plot IoU (MeanIoU)
    plt.subplot(2, 2, 4)
    plt.plot(epochs, history.history['mean_io_u'], label='Training IoU')
    plt.plot(epochs, history.history['val_mean_io_u'], label='Validation IoU')
    plt.title('Model IoU')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.xticks(tick_values)  # Set x-ticks to the desired specific values
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to plot the training history
plot_training_history(history)


In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

# Load the model from the saved .keras file
model = load_model('/content/drive/MyDrive/Advanced DL/unet_water_segmentation.keras', custom_objects={
    'Precision': Precision(),
    'Recall': Recall(),
    'MeanIoU': MeanIoU(num_classes=2)
})

# Display the model summary to confirm it loaded correctly
model.summary()

In [ ]:

# Evaluate the model on the test set
test_steps = len(image_test) // BATCH_SIZE
#test_loss, test_acc = model.evaluate(test_gen, steps=test_steps)
#print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')
# Evaluate the model on the test set
test_loss, test_acc, test_precision, test_recall, test_iou = model.evaluate(test_gen, steps=test_steps)

# Print test set results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Test IoU: {test_iou}')


# Clear memory after each epoch to avoid memory overload
gc.collect()

# Predict and visualize some test samples
preds = model.predict(test_gen, steps=test_steps)


In [ ]:
# Predict on a single batch of test data
test_batch = next(test_gen)
test_images, test_masks = test_batch
preds = model.predict(test_images)

# Plot predictions vs actual masks
n = min(5, len(test_images))  # Number of samples to display, up to 5
plt.figure(figsize=(12, 3*n))  # Adjusted figure size for a smaller grid

for i in range(n):
    # Plot the original image
    plt.subplot(n, 3, i*3 + 1)
    plt.imshow(test_images[i])
    plt.title('Original Image')
    plt.axis('off')

    # Plot the actual mask
    plt.subplot(n, 3, i*3 + 2)
    plt.imshow(test_masks[i].squeeze(), cmap='gray')
    plt.title('Actual Mask')
    plt.axis('off')

    # Plot the predicted mask
    plt.subplot(n, 3, i*3 + 3)
    plt.imshow(preds[i].squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

plt.tight_layout(pad=1.0)  # Adjust padding between subplots
plt.show()

## Final evaluation check

In [ ]:
import time
import gc
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

# Load the trained U-Net model
model = load_model('/content/drive/MyDrive/Advanced DL/Trained Models/unet_water_segmentation.keras', custom_objects={
    'Precision': Precision(),
    'Recall': Recall(),
    'MeanIoU': MeanIoU(num_classes=2)
})

# Evaluate the model on the test set
test_steps = len(image_test) // BATCH_SIZE
test_start_time = time.time()
test_loss, test_acc, test_precision, test_recall, test_iou = model.evaluate(test_gen, steps=test_steps)

test_time = time.time() - test_start_time

# Print test set results
print(f'U-Net Evaluation:')
print(f'Time: {test_time:.2f} seconds')
print(f'Average Test Precision: {test_precision:.4f}')
print(f'Average Test Recall: {test_recall:.4f}')
print(f'Average Test IoU: {test_iou:.4f}')
print(f'Average Test Accuracy: {test_acc:.4f}')

# Clear memory after evaluation
gc.collect()

# Predict and visualize some test samples
preds = model.predict(test_gen, steps=test_steps)


In [ ]:
from model.inference import load_model
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import logging
import shutil
import os
import yaml
import torch
import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from data_loader.load_data import create_datasets, create_data_loaders
from model.inference import load_model
from visualizer.visualize import visualize_data

def load_config(config_path):
    """Loads the YAML configuration file."""
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def calculate_segmentation_metrics(predictions, targets, threshold=0.5):
    """
    Calculate precision, recall, IoU, and accuracy for binary segmentation with thresholding.
    """
    # Convert model output probabilities to binary class predictions
    #predicted_probs = torch.softmax(predictions, dim=1)
    #predicted_masks = (predicted_probs[:, 1] > threshold).float()
    predicted_masks = []
    for prediction in predictions:
        mask = torch.argmax(prediction, dim=0)
        predicted_masks.append(mask)

    predicted_masks = torch.stack(predicted_masks)

    # Flatten predictions and targets for pixel-wise comparison
    targets = targets.view(-1) #torch.argmax(targets, dim=1) #.view(-1)
    predicted_masks = predicted_masks.view(-1)

    # Calculate TP, FP, TN, FN
    true_positive = (predicted_masks == 1) & (targets == 1)
    false_positive = (predicted_masks == 1) & (targets == 0)
    false_negative = (predicted_masks == 0) & (targets == 1)
    true_negative = (predicted_masks == 0) & (targets == 0)

    # Metrics
    precision = true_positive.sum().float() / (true_positive.sum() + false_positive.sum() + 1e-6)
    recall = true_positive.sum().float() / (true_positive.sum() + false_negative.sum() + 1e-6)
    iou = true_positive.sum().float() / (true_positive.sum() + false_positive.sum() + false_negative.sum() + 1e-6)
    accuracy = (true_positive.sum().float() + true_negative.sum().float()) / (targets.numel() + 1e-6)

    return precision.item(), recall.item(), iou.item(), accuracy.item()


def preprocess_image(image_path, resize):
    """Preprocesses the image for model input."""
    try:
        img = Image.open(image_path).convert('RGB')
        preprocess = transforms.Compose([
            transforms.Resize(resize),
            transforms.ToTensor()
        ])
        return preprocess(img).unsqueeze(0)  # Add batch dimension
    except Exception as e:
        logging.error(f"Error loading image {image_path}: {e}")
        return None


config_path = 'trainer_config.yaml'
config = load_config(config_path)

data_loader_config = load_config(config['DATA_LOADER_CONFIG_PATH'])
video_path = '/content/drive/MyDrive/Advanced DL/Final Video Results/Input Videos/first.mov'

cap = cv2.VideoCapture(video_path)

output_path = '/content/drive/MyDrive/Advanced DL/Final Video Results/UNET/first_output.mp4'

#Set threshold
change_threshold = 0.7

cap = cv2.VideoCapture(video_path)

# Get video details
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process each frame
frame_count = 0
previous_water_area = None
previous_frame = None
previous_mask = None

device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
model = load_model(config['MODEL_CONFIG_PATH'], device)
model = model.to(device)
pred_mask_list = []
max_idx = -1
max_val = 0
image_list = []
#pre_image_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_list.append(frame)
    cv2.imwrite('frame.jpg', frame)
    img_tensor = preprocess_image('frame.jpg', [256, 256])
    img_tensor = img_tensor.to(device)

    with torch.no_grad():
        predictions = model(img_tensor)

    if predictions.shape[1] == 2:  # Assuming binary segmentation with 2 channels
        predictions = torch.softmax(predictions, dim=1)


    threshold = 0.7
    predicted_mask = (predictions[:, 1, :, :] > threshold).long()

    predicted_mask = predicted_mask.squeeze().cpu().numpy() * 255
    predicted_mask = predicted_mask.astype(np.uint8)
    pred_mask_list.append(predicted_mask)
    current_water_area = np.sum(predicted_mask)

    if current_water_area > max_val:
        max_idx = frame_count
        max_val = current_water_area

    #if frame_count == 100:
    #    break
    print(frame_count)

    frame_count += 1

for i in range(len(pred_mask_list)):

    intersection_area = np.sum(np.logical_and(pred_mask_list[i], pred_mask_list[max_idx]))*255
    intersection_ratio = intersection_area / max_val
    current_mask = pred_mask_list[i]
    area_pixels = np.sum(current_mask)  # Count active pixels in the mask
    current_level = area_pixels / current_mask.size  # Normalize by total pixels (to get a fraction)
    current_area = area_pixels  # Area in pixels, can convert to real-world units if needed

    # Create a graphical representation of water level
    frame = image_list[i]
    height, width, _ = frame.shape
    water_line_y = int(height * (1 - (current_level)))  # Normalize height for visualization

    cv2.line(frame, (0, water_line_y), (width, water_line_y), (255, 0, 0), 5)  # Red line

    area_bar_height = int(height * (current_area / (height * width)))  # Normalize height for bar
    cv2.rectangle(frame, (width - 100, height), (width - 50, height - area_bar_height), (0, 255, 0), -1)  # Green bar

    if intersection_ratio < threshold:
            alert_text = "ALERT: Water Level Low!"
            cv2.putText(frame, alert_text, (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Red text
            red_overlay = frame.copy()
            red_overlay[:, :] = (0, 0, 255)  # Red color in BGR format

            # Blend the red overlay with the current frame
            alpha = 0.4  # Transparency level for the overlay
            frame = cv2.addWeighted(red_overlay, alpha, frame, 1 - alpha, 0)  # Blend red overlay with the frame


    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("Video segmentation completed and saved to", output_path)



In [ ]:
import cv2
import numpy as np
import os
import time
import gc
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from PIL import Image
import matplotlib.pyplot as plt

# Load the trained U-Net model
model = load_model('/content/drive/MyDrive/Advanced DL/Trained Models/unet_water_segmentation.keras', custom_objects={
    'Precision': Precision(),
    'Recall': Recall(),
    'MeanIoU': MeanIoU(num_classes=2)
})

# Load the video
video_path = '/content/drive/MyDrive/Advanced DL/Final Video Results/Input Videos/fourth.mov'
output_path = '/content/drive/MyDrive/Advanced DL/Final Video Results/UNET/fourth_output.mp4'

# Set threshold
change_threshold = 0.7

cap = cv2.VideoCapture(video_path)

# Get video details
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process each frame
frame_count = 0
previous_water_area = None
previous_mask = None
max_water_area = 0
max_mask = None
frame_list = []
pred_mask_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_list.append(frame)

    # Resize frame to match model input size (assuming model expects 256x256)
    resized_frame = cv2.resize(frame, (256, 256))

    # Preprocess the frame for model prediction
    input_frame = resized_frame / 255.0  # Normalize the image
    input_frame = np.expand_dims(input_frame, axis=0)  # Add batch dimension

    # Predict the mask
    predicted_mask = model.predict(input_frame)[0]

    # Post-process the predicted mask
    predicted_mask = (predicted_mask.squeeze() > 0.5).astype(np.uint8)  # Threshold the mask
    predicted_mask = cv2.resize(predicted_mask, (width, height))  # Resize mask to original frame size

    # Calculate the water area in the current frame
    current_water_area = np.sum(predicted_mask)
    pred_mask_list.append(predicted_mask)

    # Track the frame with the maximum water area
    if current_water_area > max_water_area:
        max_water_area = current_water_area
        max_mask = predicted_mask

    frame_count += 1
    if frame_count % 10 == 0:
        print(f'Processed {frame_count} frames...')

# Apply visualization to each frame
for i in range(len(frame_list)):
    frame = frame_list[i]
    predicted_mask = pred_mask_list[i]

    # Calculate intersection ratio
    intersection_area = np.sum(np.logical_and(predicted_mask, max_mask))
    intersection_ratio = intersection_area / (max_water_area + 1e-6)

    # Create a graphical representation of water level
    area_pixels = np.sum(predicted_mask)  # Count active pixels in the mask
    current_level = area_pixels / predicted_mask.size  # Normalize by total pixels (to get a fraction)
    current_area = area_pixels  # Area in pixels, can convert to real-world units if needed

    # Draw water level indicator
    height, width, _ = frame.shape
    water_line_y = int(height * (1 - current_level))  # Normalize height for visualization
    cv2.line(frame, (0, water_line_y), (width, water_line_y), (255, 0, 0), 5)  # Red line

    # Draw area bar indicator
    area_bar_height = int(height * (current_area / (height * width)))  # Normalize height for bar
    cv2.rectangle(frame, (width - 100, height), (width - 50, height - area_bar_height), (0, 255, 0), -1)  # Green bar

    # Add ALERT text and overlay if intersection ratio is below the threshold
    if intersection_ratio < change_threshold:
        alert_text = "ALERT: Water Level Low!"
        cv2.putText(frame, alert_text, (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Red text
        red_overlay = frame.copy()
        red_overlay[:, :] = (0, 0, 255)  # Red color in BGR format
        alpha = 0.4  # Transparency level for the overlay
        frame = cv2.addWeighted(red_overlay, alpha, frame, 1 - alpha, 0)  # Blend red overlay with the frame

    # Write the frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video segmentation completed and saved to", output_path)
